In [165]:
import requests
from master import headers
import json
import pandas as pd

url = "https://pcmap-api.place.naver.com/graphql"

# headers = {'User-Agent' : '개인 user-agent', 'Content-Type': 'application/json', 'accept': '*/*', 'accept-encoding': 'gzip, deflate, br', 'accept-language': 'ko'}
headers['Content-Type'] = "application/json"
headers['accept'] = '*/*'
headers['accept-encoding'] = 'gzip, deflate, br'
headers['accept-language'] = 'ko'
    

In [166]:
import pyproj
epsg3857 = pyproj.Proj(init='epsg:3857')
wgs84 = pyproj.Proj(init='epsg:4326')
pyproj.transform(epsg3857,wgs84,14127898.0825082,4515937.3711204)

c:\Anaconda\envs\study\lib\site-packages\pyproj\crs\crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Anaconda\envs\study\lib\site-packages\pyproj\crs\crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\강건\AppData\Local\Temp/ipykernel_19524/943088905.py:4: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  pyproj.transform

(126.91306779999996, 37.54920139999987)

In [167]:
import time
places = {}
for j in range(1,30):
    r = requests.get(url=f'https://map.naver.com/v5/api/search?caller=pcweb&query=%EC%9D%8C%EC%8B%9D%EC%A0%90&type=all&searchCoord=126.91306779999996;37.54920139999987&page={j}&displayCount=100&isPlaceRecommendationReplace=true&lang=ko', headers=headers)
    if 'error' in r.json().keys():
        print(r.json()['error']['displayMsg'])
        break
    places[j] = r.json()['result']['place']['list']
# places

잘못된 요청입니다.


In [168]:
place_id_dict = {}
for i in places.values():
    for j in i:
        place_id_dict[j['name']] = j['id']

# place_id_dict

In [169]:
place_info = []
for idx,place in enumerate(place_id_dict.values()):
    place_info.append(requests.get(url=f'https://map.naver.com/v5/api/sites/summary/{place}?lang=ko',headers=headers).json())
    # if idx == 20:
    #     break

In [ ]:
place_info[0].keys()

dict_keys(['id', 'isSite', 'name', 'type', 'rCode', 'x', 'y', 'posExact', 'address', 'fullAddress', 'addressAbbr', 'roadAddr', 'fullRoadAddress', 'phone', 'isCallLink', 'description', 'way', 'keywords', 'hasNaverTalktalkUrl', 'naverTalktalkUrl', 'adult', 'endPageUrl', 'mobileEndPageUrl', 'urlList', 'ktCallMd', 'ppc', 'images', 'imageURL', 'imageModDate', 'displayCategory', 'category', 'categories', 'categoryPaths', 'bizHour', 'bizhourInfo', 'options', 'menus', 'menuImages', 'previewImages', 'streetPanorama', 'skyPanorama', 'insidePanorama', 'interiorPanorama', 'indoorPanorama', 'entranceCoords', 'theme', 'hasNaverBooking', 'naverBookingUrl', 'naverbookingId', 'petrolInfo', 'michelinGuide', 'broadcastInfo', 'marker', 'markerSelected', 'datalab', 'reviewCount', 'dynamicData', 'markerLabel', 'isParkingSupported'])

In [ ]:
options = pd.DataFrame(place_info[0]['options'])
options['bus_id'] = place_info[0]['id']

for idx in range(1,len(place_info)):
    df = pd.DataFrame(place_info[idx]['options'])
    df['bus_id'] = place_info[idx]['id']
    options = pd.concat([options,df])

options.reset_index(drop=True,inplace=True)

options



In [ ]:
place_profile = pd.DataFrame(place_info)[['id','name','category' ,'x','y','fullAddress','fullRoadAddress','phone','description','images','imageURL','categories','bizhourInfo','menus','reviewCount']]
# place_profile

In [ ]:
# images 테이블
images_list = []
for i in range(len(place_info)):
    for j in range(len(place_info[i]['images'])):
        place_info[i]['images'][j]['bus_id'] = place_info[i]['id']
    images_list.append(place_info[i]['images'])

df = pd.DataFrame(images_list[0])

for i in range(1,len(images_list)):
    df = pd.concat([df,pd.DataFrame(images_list[i])])

df = df.reset_index(drop=True)[['bus_id','number','url','modDate']]
df

,bus_id,number,url,modDate
0,1266638776,1.0,https://ldb-phinf.pstatic.net/20220415_206/164...,2022-04-15 13:39:01
1,1266638776,2.0,https://ldb-phinf.pstatic.net/20220114_205/164...,2022-04-15 13:39:01
2,1266638776,3.0,https://ldb-phinf.pstatic.net/20220114_255/164...,2022-04-15 13:39:01
3,1266638776,4.0,https://ldb-phinf.pstatic.net/20220114_203/164...,2022-04-15 13:39:01
4,1266638776,5.0,https://ldb-phinf.pstatic.net/20220114_267/164...,2022-04-15 13:39:01
...,...,...,...,...
431,1661787502,1.0,https://ldb-phinf.pstatic.net/20220107_248/164...,2022-01-07 20:40:12
432,1661787502,2.0,https://ldb-phinf.pstatic.net/20211212_6/16392...,2022-01-07 20:40:12
433,1661787502,3.0,https://ldb-phinf.pstatic.net/20211212_30/1639...,2022-01-07 20:40:12
434,1661787502,4.0,https://ldb-phinf.pstatic.net/20211212_34/1639...,2022-01-07 20:40:12


In [ ]:
# menus
menus = pd.DataFrame(place_profile['menus'][0])
menus['bus_id'] = place_profile['id'][0]

for i in range(1,len(place_profile)):
    df = pd.DataFrame(place_profile['menus'][i])
    df['bus_id'] = place_profile['id'][i]
    menus = pd.concat([menus,df])

menus = menus[['bus_id','name','price']].reset_index(drop=True)
menus

,bus_id,name,price
0,1266638776,공감 제철 특 사시미,"62,000원"
1,1266638776,공감 제철 모듬 사시미,"39,000원"
2,1266638776,광어 + 연어 사시미,"38,000원"
3,1266638776,광어 사시미,"33,000원"
4,1266638776,치즈해물삼겹 오꼬노미야끼,"26,000원"
...,...,...,...
271,1661787502,미나리해물전,"18,000원"
272,1661787502,미나리파스타,"18,000원"
273,1661787502,가지샌드,"20,000원"
274,1661787502,바질치킨난반,"18,000원"


In [ ]:
# 식당 기본 상세정보
place_default_profile = place_profile[['id','name','category','x','y','fullAddress','fullRoadAddress','phone','description','imageURL','bizhourInfo','reviewCount']].copy()
# place_default_profile.reset_index(drop=True,inplace=True)

In [ ]:
place_default_profile.rename(columns={'id':'busId','name':'restaurantName','category':'restaurantCategory','phone':'tellNumber','bizhourInfo':'businessHourInfo'},inplace=True)

In [ ]:
place_default_profile['siCode'] = place_default_profile['fullAddress'].apply(lambda x: x.split(" ")[0])
place_default_profile['guCode'] = place_default_profile['fullAddress'].apply(lambda x: x.split(" ")[1])
place_default_profile['dongCode'] = place_default_profile['fullAddress'].apply(lambda x: x.split(" ")[2])

# place_default_profile

In [ ]:
import json
payload = [{f"{k}":f"{v}" for k, v in row.items()} for _,row in place_default_profile.iterrows()]

for row in payload:
    requests.post(headers=headers, url = "http://localhost:8080/restaurant/createRestaurantInfo",data=json.dumps(row, indent=4, ensure_ascii=False).encode('utf-8'))

In [ ]:

# for row in payload:
#     print(json.dumps(row, indent=4, ensure_ascii=False))

